In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from functools import reduce

#Load workforce data
workforce_path = r"C:\Users\babak\Downloads\Final Project\health-workforce-canada-2019-2023-overview-data-tables-en.xlsx"
workforce_data = {}
for i in range(6, 15):
    table_name = f"Table {i}"
    workforce_data[table_name] = pd.read_excel(workforce_path, sheet_name=table_name)

#Extract Doctor and NP counts
def extract_counts(df, province, count_col):
    df.columns = df.iloc[0]
    df = df[1:]
    df = df.rename(columns={df.columns[0]: "Type of professional"})
    df["Type of professional"] = df["Type of professional"].ffill()
    df_2022 = df[df["Year"] == 2022]

    def parse_count(row):
        try:
            return int(str(row[count_col]).replace("‡", "").replace("*", "").replace("—", "").strip())
        except:
            return None

    np_row = df_2022[df_2022["Type of professional"].str.contains("nurse practitioner", case=False, na=False)]
    doc_row = df_2022[df_2022["Type of professional"].str.lower() == "physicians"]

    np_count = parse_count(np_row.iloc[0]) if not np_row.empty else None
    doc_count = parse_count(doc_row.iloc[0]) if not doc_row.empty else None

    return {
        "Province": province,
        "Doctors": doc_count,
        "Nurse Practitioners": np_count,
        "Doctor-to-NP Ratio": doc_count / np_count if doc_count and np_count else None
    }

province_cols = {
    "Table 6": ("Quebec", "Quebec: Count"),
    "Table 7": ("Ontario", "Ontario: Count"),
    "Table 8": ("Manitoba", "Manitoba: Count"),
    "Table 9": ("Saskatchewan", "Saskatchewan: Count"),
    "Table 10": ("Alberta", "Alberta: Count"),
    "Table 11": ("British Columbia", "British Columbia: Count"),
    "Table 12": ("Yukon", "Yukon: Count"),
    "Table 13": ("Northwest Territories", "Northwest Territories: Count"),
    "Table 14": ("Nunavut", "Nunavut: Count")
}

doctor_np_data = [extract_counts(workforce_data[k], v[0], v[1]) for k, v in province_cols.items()]
doctor_np_df = pd.DataFrame(doctor_np_data)

#Load CMWF 2019 access data
cmwf_2019_path = r"C:\Users\babak\Downloads\Final Project\cmwf-2019-data-tables-en.xlsx"
access_data = {}
for i in range(12, 18):
    sheet = f"{i} Access to care"
    access_data[sheet] = pd.read_excel(cmwf_2019_path, sheet_name=sheet)

def extract_province_table(df, start_row=18):
    df_clean = df.iloc[start_row:].dropna(how='all').reset_index(drop=True)
    df_clean.columns = df_clean.iloc[0]
    return df_clean[1:]

province_map = {
    'N.L.': 'Newfoundland and Labrador',
    'P.E.I.': 'Prince Edward Island',
    'N.S.': 'Nova Scotia',
    'N.B.': 'New Brunswick',
    'Que.': 'Quebec',
    'Ont.': 'Ontario',
    'Man.': 'Manitoba',
    'Sask.': 'Saskatchewan',
    'Alta.': 'Alberta',
    'B.C.': 'British Columbia',
    'Y.T.': 'Yukon',
    'N.W.T.': 'Northwest Territories',
    'Nvt.': 'Nunavut'
}

access_tables = []
for name, df in access_data.items():
    cleaned = extract_province_table(df)
    cleaned.columns = ['Province'] + [f"{name}_Q{i+1}" for i in range(len(cleaned.columns)-1)]
    cleaned['Province'] = cleaned['Province'].map(province_map)
    access_tables.append(cleaned)

access_df = reduce(lambda left, right: pd.merge(left, right, on="Province", how="outer"), access_tables)
merged_df = pd.merge(access_df, doctor_np_df, on="Province", how="inner")
merged_df_clean = merged_df.dropna(subset=["Doctor-to-NP Ratio"])

#Define metrics to analyze
correlation_targets = {
    "Same/Next-Day Appointments": "12 Access to care_Q1",
    "After-Hours Care (After 6 PM)": "12 Access to care_Q1",  # Same metric
    "Weekend Availability": "13 Access to care_Q1",
    "Home Visits": "17 Access to care_Q1",
    "Phone/Email Consultations": "16 Access to care_Q1",
    "Shorter Wait Times": "15 Access to care_Q1",
    "Fewer ED Visits (Minor Issues)": "14 Access to care_Q1"
}

#Analyze correlation + linear regression
regression_results = []

for label, column in correlation_targets.items():
    if column not in merged_df_clean.columns:
        continue

    df = merged_df_clean[["Doctor-to-NP Ratio", column, "Province"]].dropna()
    X = df[["Doctor-to-NP Ratio"]].astype(float).values
    y = df[column].astype(float).values

    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)

    slope = model.coef_[0]
    intercept = model.intercept_
    r_squared = model.score(X, y)

    regression_results.append({
        "Metric": label,
        "Slope": slope,
        "Intercept": intercept,
        "R²": r_squared
    })

    # Plot the regression
    plt.figure(figsize=(8, 6))
    plt.scatter(X, y, label="Actual")
    plt.plot(X, y_pred, color="red", label="Regression Line")
    for _, row in df.iterrows():
        plt.text(row["Doctor-to-NP Ratio"] + 0.2, row[column], row["Province"], fontsize=9)
    plt.xlabel("Doctor-to-NP Ratio")
    plt.ylabel(f"{label} (%)")
    plt.title(f"Doctor-to-NP Ratio vs. {label}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

#Summary of results 
regression_summary_df = pd.DataFrame(regression_results)
print("\nRegression Summary:")
print(regression_summary_df)
